In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

판다스는 데이터셋을 자연스럽게 나누고 요약하는 groupby라는 유연한 방법을 제공

# 그룹 연산에 대한 고찰

데이터를 'key1'으로 묶고 각 그룹에서 data1의 평균을 구한다

In [3]:
df = pd.DataFrame({
    'key1': ['a', 'a', None, 'b', 'b', 'a', None],
    'key2': pd.Series([1, 2, 1, 2, 1, None, 1], dtype='Int64'),
    'data1': np.random.standard_normal(7),
    'data2': np.random.standard_normal(7)
})

df

grouped = df.groupby('key1')['data1']
grouped

grouped.mean()

,key1,key2,data1,data2
0,a,1,0.904755,0.454984
1,a,2,0.373847,0.870499
2,None,1,0.151460,0.637315
3,b,2,0.569420,1.445859
4,b,1,-0.159645,-1.502243
5,a,<NA>,0.822559,0.567236
6,None,1,-0.075261,0.112971


key1
a    0.700387
b    0.204887
Name: data1, dtype: float64

여러 개의 배열을 리스트로

데이터를 두 개의 색인으로 묶었고, 그 결과 계층적 색인을 갖는 Series를 얻을 수 있다

In [4]:
means = df.groupby(['key1', 'key2'])['data1'].mean()
means

means.unstack()

key1  key2
a     1       0.904755
      2       0.373847
b     1      -0.159645
      2       0.569420
Name: data1, dtype: float64

key2,1,2
key1,,
a,0.904755,0.373847
b,-0.159645,0.569420


In [5]:
states = np.array(['oh', 'ca', 'ca', 'oh', 'oh', 'ca', 'oh'])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

df.groupby([states, years])['data1'].mean()

df.groupby([states, years])['data1'].mean().unstack()

ca  2005    0.598203
    2006    0.151460
oh  2005    0.737088
    2006   -0.117453
Name: data1, dtype: float64

,2005,2006
ca,0.598203,0.151460
oh,0.737088,-0.117453


In [18]:
df.groupby('key1').mean()

df.groupby('key2').mean(numeric_only = True) # numeric_only가 없으면 오류가 발생한다 
# => 결과에 key1이 빠진 이유는 숫자 데이터가 아니기 때문이다

df.groupby(['key1', 'key2']).mean()

,key2,data1,data2
key1,,,
a,1.5,-0.702969,-0.493307
b,1.5,0.105672,0.410283


,data1,data2
key2,,
1,0.511322,0.201156
2,-0.199798,-0.436169


data1     data2
key1 key2                    
a    1    -0.276318  0.045991
     2    -0.490218 -0.806031
b    1     0.120722  0.886872
     2     0.090622 -0.066306

In [20]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

그룹 색인에서 누락된 값은 기본적으로 결과에서 제외된다

In [22]:
df.groupby('key1', dropna = False).size() # dropna = False를 사용하여 비활성화 할 수 있다
df.groupby(['key1', 'key2'], dropna = False).size()

key1
a      3
b      2
NaN    2
dtype: int64

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [24]:
df.groupby('key1').count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


## 그룹 간 순회하기

groupby 메서드에서 반환된 객체는 순회(이터레이션)를 지원하는데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환

In [29]:
for name, group in df.groupby('key1'):
    print(name)
    print(group, '\n')

a
  key1  key2     data1     data2
0    a     1 -0.276318  0.045991
1    a     2 -0.490218 -0.806031
5    a  <NA> -1.342372 -0.719882 

b
  key1  key2     data1     data2
3    b     2  0.090622 -0.066306
4    b     1  0.120722  0.886872 



In [21]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group, '\n')

('a', 1)
  key1  key2     data1     data2
0    a     1  0.904755  0.454984 

('a', 2)
  key1  key2     data1     data2
1    a     2  0.373847  0.870499 

('b', 1)
  key1  key2     data1     data2
4    b     1 -0.159645 -1.502243 

('b', 2)
  key1  key2    data1     data2
3    b     2  0.56942  1.445859 



In [23]:
# 그룹별 데이터를 딕셔너리형으로 변환

dic = {key: group for key, group in df.groupby('key1')}
dic

print()
dic['b']

{'a':   key1  key2     data1     data2
 0    a     1  0.904755  0.454984
 1    a     2  0.373847  0.870499
 5    a  <NA>  0.822559  0.567236,
 'b':   key1  key2     data1     data2
 3    b     2  0.569420  1.445859
 4    b     1 -0.159645 -1.502243}

,key1,key2,data1,data2
3,b,2,0.569420,1.445859
4,b,1,-0.159645,-1.502243


In [24]:
df

,key1,key2,data1,data2
0,a,1,0.904755,0.454984
1,a,2,0.373847,0.870499
2,None,1,0.151460,0.637315
3,b,2,0.569420,1.445859
4,b,1,-0.159645,-1.502243
5,a,<NA>,0.822559,0.567236
6,None,1,-0.075261,0.112971


In [25]:
grouped = df.groupby(
    {
        'key1': 'key',
        'key2': 'key',
        'data1': 'data',
        'data2': 'data'
    },
    axis='columns')  #??????????????

for gb_key, gb_val in grouped:
    print(gb_key)
    print(gb_val, '\n')

data
      data1     data2
0  0.904755  0.454984
1  0.373847  0.870499
2  0.151460  0.637315
3  0.569420  1.445859
4 -0.159645 -1.502243
5  0.822559  0.567236
6 -0.075261  0.112971 

key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1 



## 열이나 열의 일부만 선택하기

In [39]:
df.groupby('key1')['data1']
df.groupby('key1')[['data1']]

df['data1'].groupby(df['key1'])
df['data2'].groupby(df['key1'])

data2열에 대해서만 평균을 구하고 결과를 DataFrame으로 받기

In [40]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    1     0.045991
     2    -0.806031
b    1     0.886872
     2    -0.066306

In [42]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

s_grouped.mean()

key1  key2
a     1       0.045991
      2      -0.806031
b     1       0.886872
      2      -0.066306
Name: data2, dtype: float64

## 딕셔너리와 Series에서 그룹화하기

In [43]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['joe', 'steve', 'wanda', 'jill', 'trey'])

people.iloc[2:3, [1, 2]] = np.nan  # nan 추가
people

,a,b,c,d,e
joe,-0.514726,-1.699243,-1.086434,-1.289833,-0.066582
steve,0.192764,1.091143,0.854986,-0.293514,1.700077
wanda,2.134331,NaN,NaN,-0.374330,1.034991
jill,-0.597286,-0.104638,0.488009,-1.272237,1.207263
trey,-0.189179,-0.381722,-1.118771,0.687922,0.949635


In [47]:
mapping = {
    'a': 'red',
    'b': 'red',
    'c': 'blue',
    'd': 'blue',
    'e': 'red',
    'f': 'orange'
}  #??????????????????

by_column = people.groupby(mapping, axis='columns')
by_column.sum()

,blue,red
joe,-2.376267,-2.280551
steve,0.561472,2.983984
wanda,-0.374330,3.169322
jill,-0.784228,0.505339
trey,-0.430849,0.378733


In [49]:
map_series = pd.Series(mapping)
map_series

people.groupby(map_series, axis = 1).count()

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

,blue,red
joe,2,3
steve,2,3
wanda,1,2
jill,2,3
trey,2,3


## 함수로 그룹화하기

In [50]:
people.groupby(len).sum()  #????????????

,a,b,c,d,e
3,-0.514726,-1.699243,-1.086434,-1.289833,-0.066582
4,-0.786465,-0.486360,-0.630761,-0.584316,2.156898
5,2.327095,1.091143,0.854986,-0.667844,2.735068


In [52]:
key_lst = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_lst]).min()

,,a,b,c,d,e
3,one,-0.514726,-1.699243,-1.086434,-1.289833,-0.066582
4,two,-0.597286,-0.381722,-1.118771,-1.272237,0.949635
5,one,0.192764,1.091143,0.854986,-0.374330,1.034991


## 색인 단계로 그룹화하기

In [56]:
columns = pd.MultiIndex.from_arrays(
    [['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

hier_df.groupby(level='cty', axis=1).count()

cty          US                            JP          
tenor         1         3         5         1         3
0      1.588490  1.635568 -0.042285  0.749100 -0.580845
1      0.815664 -2.217259  1.804044  0.453913  1.051388
2     -0.121527  0.056423 -1.561273 -0.327431  0.111845
3     -0.965608 -1.638146 -1.245223 -1.138844 -1.824149

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 데이터 집계

In [33]:
grouped = df.groupby('key1')

for key, group in grouped['data1']:
    print(key)
    print(group, '\n')

grouped['data1'].nsmallest(1)
grouped['data1'].nsmallest(2)

a
0    0.904755
1    0.373847
5    0.822559
Name: data1, dtype: float64 

b
3    0.569420
4   -0.159645
Name: data1, dtype: float64 



key1   
a     1    0.373847
b     4   -0.159645
Name: data1, dtype: float64

key1   
a     1    0.373847
      5    0.822559
b     4   -0.159645
      3    0.569420
Name: data1, dtype: float64

In [34]:
def peak_to_peak(arr):
    return arr.max() - arr.min()


grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,0.530908,0.415516
b,1,0.729065,2.948101


In [42]:
grouped.describe()
grouped.describe()['key2']['count']

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.700387  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.204887  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.863657  0.904755   3.0  0.630906  0.214951  0.454984  0.511110   
b     0.387154  0.569420   2.0 -0.028192  2.084622 -1.502243 -0.765217   

                                    
           50%       75%       max  
key1                                
a     0.567236  0.718868  0.870499  
b    -0.028192  0.708833  1.445859  

[2 rows x 24 columns]

key1
a    2.0
b    2.0
Name: count, dtype: Float64

## 열에 여러 가지 함수 적용하기

In [2]:
tips = pd.read_csv('tips.csv')
tips.head(3)

tips['tip_pct'] = tips['tip'] / (tips['total_bill'] - tips['tip'])
tips.head(3)

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3


,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.063204
1,10.34,1.66,No,Sun,Dinner,3,0.191244
2,21.01,3.50,No,Sun,Dinner,3,0.199886


In [3]:
# grouped_pct = tips.groupby(['day', 'smoker'])['tip_pct'].mean()
# grouped_pct.unstack()

grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.179740
      Yes       0.216293
Sat   No        0.190412
      Yes       0.179833
Sun   No        0.193617
      Yes       0.322021
Thur  No        0.193424
      Yes       0.198508
Name: tip_pct, dtype: float64

In [5]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [6]:
grouped_pct.agg(['mean', 'std', peak_to_peak])
grouped_pct.agg([np.mean, np.std, peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.179740  0.039458      0.094263
     Yes     0.216293  0.077530      0.242219
Sat  No      0.190412  0.058626      0.352192
     Yes     0.179833  0.089496      0.446137
Sun  No      0.193617  0.060302      0.274897
     Yes     0.322021  0.538061      2.382107
Thur No      0.193424  0.056065      0.284273
     Yes     0.198508  0.057170      0.219047

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.179740  0.039458      0.094263
     Yes     0.216293  0.077530      0.242219
Sat  No      0.190412  0.058626      0.352192
     Yes     0.179833  0.089496      0.446137
Sun  No      0.193617  0.060302      0.274897
     Yes     0.322021  0.538061      2.382107
Thur No      0.193424  0.056065      0.284273
     Yes     0.198508  0.057170      0.219047

In [7]:
grouped_pct.agg([('평균', 'mean'), ('표준편차', 'std')])

평균      표준편차
day  smoker                    
Fri  No      0.179740  0.039458
     Yes     0.216293  0.077530
Sat  No      0.190412  0.058626
     Yes     0.179833  0.089496
Sun  No      0.193617  0.060302
     Yes     0.322021  0.538061
Thur No      0.193424  0.056065
     Yes     0.198508  0.057170

In [8]:
func = ['count', 'mean', 'max']

res = grouped[['tip_pct', 'total_bill']].agg(func)
res

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.179740  0.231125          4  18.420000  22.75
     Yes         15  0.216293  0.357737         15  16.813333  40.17
Sat  No          45  0.190412  0.412409         45  19.661778  48.33
     Yes         42  0.179833  0.483092         42  21.276667  50.81
Sun  No          57  0.193617  0.338101         57  20.506667  48.17
     Yes         19  0.322021  2.452381         19  24.120000  45.35
Thur No          45  0.193424  0.362976         45  17.113111  41.19
     Yes         17  0.198508  0.317965         17  19.190588  43.11

In [9]:
res['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.179740  0.231125
     Yes        15  0.216293  0.357737
Sat  No         45  0.190412  0.412409
     Yes        42  0.179833  0.483092
Sun  No         57  0.193617  0.338101
     Yes        19  0.322021  2.452381
Thur No         45  0.193424  0.362976
     Yes        17  0.198508  0.317965

In [10]:
f_tup = [('Average', 'mean'), ('Variance', np.var)]

grouped[['tip_pct', 'total_bill']].agg(f_tup)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.179740  0.001557  18.420000   25.596333
     Yes     0.216293  0.006011  16.813333   82.562438
Sat  No      0.190412  0.003437  19.661778   79.908965
     Yes     0.179833  0.008010  21.276667  101.387535
Sun  No      0.193617  0.003636  20.506667   66.099980
     Yes     0.322021  0.289509  24.120000  109.046044
Thur No      0.193424  0.003143  17.113111   59.625081
     Yes     0.198508  0.003268  19.190588   69.808518

In [11]:
grouped.agg({
    'tip' : np.max, 
    'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [12]:
grouped.agg({
    'tip_pct' : ['min', 'max', 'mean', 'std'], 
    'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.136861  0.231125  0.179740  0.039458    9
     Yes     0.115518  0.357737  0.216293  0.077530   31
Sat  No      0.060217  0.412409  0.190412  0.058626  115
     Yes     0.036955  0.483092  0.179833  0.089496  104
Sun  No      0.063204  0.338101  0.193617  0.060302  167
     Yes     0.070274  2.452381  0.322021  0.538061   49
Thur No      0.078704  0.362976  0.193424  0.056065  112
     Yes     0.098918  0.317965  0.198508  0.057170   40

## 색인되지 않은 형태로 집계된 데이터 반환하기

In [35]:
grouped = tips.groupby(['day', 'smoker'], as_index = False)
grouped.mean(numeric_only = True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.179740
1,Fri,Yes,16.813333,2.714000,2.066667,0.216293
2,Sat,No,19.661778,3.102889,2.555556,0.190412
3,Sat,Yes,21.276667,2.875476,2.476190,0.179833
4,Sun,No,20.506667,3.167895,2.929825,0.193617
5,Sun,Yes,24.120000,3.516842,2.578947,0.322021
6,Thur,No,17.113111,2.673778,2.488889,0.193424
7,Thur,Yes,19.190588,3.030000,2.352941,0.198508


# apply 매서드 : 일반적인 분리 - 적용 - 병합

In [17]:
# 데이터프레임 tip_pct에서 큰 캆 5개의 행을 뽑아라

def top(df, n = 5, column = 'tip_pct'):
    return df.sort_values(column, ascending = False)[:n]

top(tips, n = 6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,2.452381
178,9.60,4.00,Yes,Sun,Dinner,2,0.714286
67,3.07,1.00,Yes,Sat,Dinner,1,0.483092
232,11.61,3.39,No,Sat,Dinner,2,0.412409
183,23.17,6.50,Yes,Sun,Dinner,4,0.389922
109,14.31,4.00,Yes,Sat,Dinner,2,0.387973


In [20]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.412409
       149        7.51  2.00     No  Thur   Lunch     2  0.362976
       51        10.29  2.60     No   Sun  Dinner     2  0.338101
       185       20.69  5.00     No   Sun  Dinner     5  0.318674
       88        24.71  5.85     No  Thur   Lunch     2  0.310180
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  2.452381
       178        9.60  4.00    Yes   Sun  Dinner     2  0.714286
       67         3.07  1.00    Yes   Sat  Dinner     1  0.483092
       183       23.17  6.50    Yes   Sun  Dinner     4  0.389922
       109       14.31  4.00    Yes   Sat  Dinner     2  0.387973

In [23]:
tips.groupby(['smoker', 'day']).apply(top, n = 1, column = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.166667
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.228833
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.115821
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.138160
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.133465
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.245038
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.083632
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.131199

In [29]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

result.unstack()

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.192237,0.057665,0.060217,0.158622,0.184308,0.227015,0.412409
Yes,93.0,0.218176,0.254295,0.036955,0.119534,0.181818,0.242326,2.452381


       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.192237
       Yes         0.218176
std    No          0.057665
       Yes         0.254295
min    No          0.060217
       Yes         0.036955
25%    No          0.158622
       Yes         0.119534
50%    No          0.184308
       Yes         0.181818
75%    No          0.227015
       Yes         0.242326
max    No          0.412409
       Yes         2.452381
dtype: float64

## 그룹 키 생략하기

In [36]:
tips.groupby('smoker').apply(top)

tips.groupby('smoker', as_index = False).apply(top)

tips.groupby('smoker', group_keys = False).apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.412409
       149        7.51  2.00     No  Thur   Lunch     2  0.362976
       51        10.29  2.60     No   Sun  Dinner     2  0.338101
       185       20.69  5.00     No   Sun  Dinner     5  0.318674
       88        24.71  5.85     No  Thur   Lunch     2  0.310180
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  2.452381
       178        9.60  4.00    Yes   Sun  Dinner     2  0.714286
       67         3.07  1.00    Yes   Sat  Dinner     1  0.483092
       183       23.17  6.50    Yes   Sun  Dinner     4  0.389922
       109       14.31  4.00    Yes   Sat  Dinner     2  0.387973

total_bill   tip smoker   day    time  size   tip_pct
0 232       11.61  3.39     No   Sat  Dinner     2  0.412409
  149        7.51  2.00     No  Thur   Lunch     2  0.362976
  51        10.29  2.60     No   Sun  Dinner     2  0.338101
  185       20.69  5.00     No   Sun  Dinner     5  0.318674
  88        24.71  5.85     No  Thur   Lunch     2  0.310180
1 172        7.25  5.15    Yes   Sun  Dinner     2  2.452381
  178        9.60  4.00    Yes   Sun  Dinner     2  0.714286
  67         3.07  1.00    Yes   Sat  Dinner     1  0.483092
  183       23.17  6.50    Yes   Sun  Dinner     4  0.389922
  109       14.31  4.00    Yes   Sat  Dinner     2  0.387973

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.412409
149,7.51,2.00,No,Thur,Lunch,2,0.362976
51,10.29,2.60,No,Sun,Dinner,2,0.338101
185,20.69,5.00,No,Sun,Dinner,5,0.318674
88,24.71,5.85,No,Thur,Lunch,2,0.310180
172,7.25,5.15,Yes,Sun,Dinner,2,2.452381
178,9.60,4.00,Yes,Sun,Dinner,2,0.714286
67,3.07,1.00,Yes,Sat,Dinner,1,0.483092
183,23.17,6.50,Yes,Sun,Dinner,4,0.389922
109,14.31,4.00,Yes,Sat,Dinner,2,0.387973


## 사분위수 분석과 버킷 분석

In [37]:
frame = pd.DataFrame({'data1' : np.random.standard_normal(1000),
                      'data2' : np.random.standard_normal(1000)})

frame.head()

,data1,data2
0,-1.696638,1.957290
1,0.359726,-0.574149
2,-0.212645,-0.458751
3,-1.523147,-0.485868
4,1.102797,0.338700


In [39]:
quartiles = pd.cut(frame['data1'], 4)
quartiles.head(10)

0     (-3.281, -1.673]
1      (-0.0717, 1.53]
2    (-1.673, -0.0717]
3    (-1.673, -0.0717]
4      (-0.0717, 1.53]
5    (-1.673, -0.0717]
6      (-0.0717, 1.53]
7    (-1.673, -0.0717]
8      (-0.0717, 1.53]
9    (-1.673, -0.0717]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.281, -1.673] < (-1.673, -0.0717] < (-0.0717, 1.53] < (1.53, 3.131]]

In [40]:
def get_stats(group):
    return pd.DataFrame(
        {'min' : group.min(), 'max' : group.max(),
         'count' : group.count(), 'mean' : group.mean()})

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

min       max  count      mean
data1                                                       
(-3.281, -1.673]  data1 -3.274639 -1.675700     49 -2.062484
                  data2 -1.750611  2.349222     49 -0.209430
(-1.673, -0.0717] data1 -1.650808 -0.071919    436 -0.685550
                  data2 -3.327283  2.827209    436 -0.033751
(-0.0717, 1.53]   data1 -0.057923  1.513605    454  0.652752
                  data2 -2.734871  2.708917    454 -0.075075
(1.53, 3.131]     data1  1.537449  3.131280     61  1.983311
                  data2 -1.694104  2.200512     61  0.047287

In [41]:
grouped.agg(['min', 'max', 'count', 'mean'])

data1                               data2            \
                        min       max count      mean       min       max   
data1                                                                       
(-3.281, -1.673]  -3.274639 -1.675700    49 -2.062484 -1.750611  2.349222   
(-1.673, -0.0717] -1.650808 -0.071919   436 -0.685550 -3.327283  2.827209   
(-0.0717, 1.53]   -0.057923  1.513605   454  0.652752 -2.734871  2.708917   
(1.53, 3.131]      1.537449  3.131280    61  1.983311 -1.694104  2.200512   

                                   
                  count      mean  
data1                              
(-3.281, -1.673]     49 -0.209430  
(-1.673, -0.0717]   436 -0.033751  
(-0.0717, 1.53]     454 -0.075075  
(1.53, 3.131]        61  0.047287

In [43]:
quartiles_samp = pd.qcut(frame['data1'], 4, labels = False)
quartiles_samp.head()

grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

0    0
1    2
2    1
3    0
4    3
Name: data1, dtype: int64

min       max  count      mean
data1                                           
0     data1 -3.274639 -0.688977    250 -1.260408
      data2 -3.327283  2.408496    250 -0.069383
1     data1 -0.684991  0.007887    250 -0.340783
      data2 -2.734635  2.827209    250 -0.028997
2     data1  0.009717  0.749360    250  0.372687
      data2 -2.734871  2.708917    250 -0.107802
3     data1  0.749486  3.131280    250  1.297983
      data2 -2.649497  2.200512    250 -0.018526

## 그룹별 값으로 결측치 채우기

In [45]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

s.fillna(s.mean())

0         NaN
1   -1.406674
2         NaN
3    0.048655
4         NaN
5   -0.369544
dtype: float64

0   -0.575854
1   -1.406674
2   -0.575854
3    0.048655
4   -0.575854
5   -0.369544
dtype: float64

In [49]:
stats = ['ohoi', 'new york', 'vermont', 'florida', 'oregon', 'nevada', 'california', 'idaho']
group_key = ['east', 'east', 'east', 'east', 'west', 'west', 'west', 'west']

data = pd.Series(np.random.standard_normal(8), index = stats)
data

data[['vermont', 'nevada', 'idaho']] = np.nan
data

data.groupby(group_key).size()
data.groupby(group_key).count() # nan값은 빼고 count
data.groupby(group_key).mean()

ohoi          0.300207
new york     -1.241284
vermont       0.054178
florida      -1.176714
oregon        1.996854
nevada        0.922216
california   -0.348162
idaho         1.674705
dtype: float64

ohoi          0.300207
new york     -1.241284
vermont            NaN
florida      -1.176714
oregon        1.996854
nevada             NaN
california   -0.348162
idaho              NaN
dtype: float64

east    4
west    4
dtype: int64

east    3
west    2
dtype: int64

east   -0.705930
west    0.824346
dtype: float64

In [53]:
def fill_mean(group):
    return group.fillna(group.mean())


data.groupby(group_key).apply(fill_mean)
data.groupby(group_key, group_keys=False).apply(fill_mean)

east  ohoi          0.300207
      new york     -1.241284
      vermont      -0.705930
      florida      -1.176714
west  oregon        1.996854
      nevada        0.824346
      california   -0.348162
      idaho         0.824346
dtype: float64

ohoi          0.300207
new york     -1.241284
vermont      -0.705930
florida      -1.176714
oregon        1.996854
nevada        0.824346
california   -0.348162
idaho         0.824346
dtype: float64

In [55]:
fill_values = {'east' : 0.5, 'west' : -1}

def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

east  ohoi          0.300207
      new york     -1.241284
      vermont       0.500000
      florida      -1.176714
west  oregon        1.996854
      nevada       -1.000000
      california   -0.348162
      idaho        -1.000000
dtype: float64

## 랜덤 표본과 순열

In [59]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4 # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10,...(반복 3번)
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q'] # ['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'K', 'Q']

cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index = cards)

deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [63]:
def draw(deck, n = 5):
    return deck.sample(n) # deck 객체에서 n개를 추출

draw(deck)

8C     8
4S     4
KH    10
JH    10
9C     9
dtype: int64

In [71]:
def get_suit(card):
    return card[-1] # 마지막 글자는 모양 표시

deck.groupby(get_suit).apply(draw, n = 2) # groupby로 C, D, H, S별로 2장씩 카드를 뽑는다
deck.groupby(get_suit, group_keys = False).apply(draw, n = 2)

C  AC     1
   JC    10
D  3D     3
   8D     8
H  JH    10
   3H     3
S  3S     3
   AS     1
dtype: int64

3C      3
10C    10
KD     10
9D      9
8H      8
5H      5
3S      3
2S      2
dtype: int64

## 그룹 가중평균과 상관관계

In [73]:
df = pd.DataFrame({
    'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
    'data': np.random.standard_normal(8),
    'weights': np.random.uniform(size=8)
})

df

,category,data,weights
0,a,0.822685,0.204826
1,a,-1.771005,0.899619
2,a,0.548833,0.904066
3,a,-0.866645,0.980447
4,b,-1.420950,0.475941
5,b,-0.391774,0.321838
6,b,0.401239,0.165464
7,b,0.491210,0.455150


In [76]:
grouped = df.groupby('category')

def get_wavg(group): # category별 그룹 가중평균
    return np.average(group['data'], weights = group['weights'])

grouped.apply(get_wavg)

category
a   -0.594937
b   -0.361262
dtype: float64

In [78]:
close_px = pd.read_csv('stock_px.csv', parse_dates = True, index_col = 0)\

close_px.info()
close_px.tail(4)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [79]:
# spx 지수와의 상관관계를 살펴보기

def spx_corr(group):
    return group.corrwith(group['SPX'])

In [80]:
rets = close_px.pct_change().dropna()

In [82]:
def get_year(x):
    return x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [83]:
def corr_aapl_msft(group):
    return group['AAPL'].corr(group['MSFT'])

by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## 그룹별 선형 회귀

In [88]:
import statsmodels.api as sm
def regress(data, yvar = None, xvars = None):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [89]:
by_year.apply(regress, yvar = 'AAPL', xvars = ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


In [90]:
import sys
sys.version

'3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]'

# 그룹 변환과 래핑되지 않은 groupby

In [91]:
df = pd.DataFrame({'key' : ['a', 'b', 'c'] * 4,
                   'value' : np.arange(12.)})

df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [92]:
g = df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [97]:
def get_mean(group):
    return group.mean()

g.transform(get_mean)
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [99]:
def times_two(group):
    return group * 2

g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [101]:
def get_ranks(group):
    return group.rank(ascending = False)

g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [104]:
def normalize(x):
    return (x - x.mean()) / x.std()

g.transform(normalize)
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [106]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

# 피벗테이블과 교차표

In [107]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.063204
1,10.34,1.66,No,Sun,Dinner,3,0.191244
2,21.01,3.50,No,Sun,Dinner,3,0.199886
3,23.68,3.31,No,Sun,Dinner,2,0.162494
4,24.59,3.61,No,Sun,Dinner,4,0.172069


In [109]:
tips.pivot_table(index = ['day', 'smoker'],
                 values = ['size', 'tip', 'tip_pct', 'total_bill'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.179740   18.420000
     Yes     2.066667  2.714000  0.216293   16.813333
Sat  No      2.555556  3.102889  0.190412   19.661778
     Yes     2.476190  2.875476  0.179833   21.276667
Sun  No      2.929825  3.167895  0.193617   20.506667
     Yes     2.578947  3.516842  0.322021   24.120000
Thur No      2.488889  2.673778  0.193424   17.113111
     Yes     2.352941  3.030000  0.198508   19.190588

In [110]:
tips.pivot_table(index = ['time', 'day'], columns = 'smoker',
                 values = ['tip_pct', 'size'])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.162612  0.202545
       Sat   2.555556  2.476190  0.190412  0.179833
       Sun   2.929825  2.578947  0.193617  0.322021
       Thur  2.000000       NaN  0.190114       NaN
Lunch  Fri   3.000000  1.833333  0.231125  0.236915
       Thur  2.500000  2.352941  0.193499  0.198508

In [111]:
# margins = True : 행과 열에 ALL이 추가되고 단일 줄 안에서 그룹 통계를 얻을 수 있다

tips.pivot_table(index = ['time', 'day'], columns = 'smoker', 
                values = ['tip_pct', 'size'], margins = True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.162612  0.202545  0.192562
       Sat   2.555556  2.476190  2.517241  0.190412  0.179833  0.185305
       Sun   2.929825  2.578947  2.842105  0.193617  0.322021  0.225718
       Thur  2.000000       NaN  2.000000  0.190114       NaN  0.190114
Lunch  Fri   3.000000  1.833333  2.000000  0.231125  0.236915  0.236088
       Thur  2.500000  2.352941  2.459016  0.193499  0.198508  0.194895
All          2.668874  2.408602  2.569672  0.192237  0.218176  0.202123

In [112]:
tips.pivot_table(index = ['time', 'smoker'], columns = 'day',
                 values = 'tip_pct', aggfunc = len, margins = True) # len은 count와 다르게 nan도 같이 출력

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [113]:
tips.pivot_table(index = ['time', 'size', 'smoker'], columns = 'day',
                 values = 'tip_pct', fill_value = 0) # nan값을 0으로 대체

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.160000  0.000000  0.000000
            Yes     0.000000  0.483092  0.000000  0.000000
       2    No      0.162612  0.198319  0.206535  0.190114
            Yes     0.211180  0.178877  0.400522  0.000000
       3    No      0.000000  0.183870  0.182962  0.000000
            Yes     0.000000  0.176599  0.183278  0.000000
       4    No      0.000000  0.177734  0.175289  0.000000
            Yes     0.133465  0.147074  0.254373  0.000000
       5    No      0.000000  0.000000  0.263344  0.000000
            Yes     0.000000  0.119284  0.070274  0.000000
       6    No      0.000000  0.000000  0.115821  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.222087
            Yes     0.288288  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.201503
            Yes     0.226641  0.000000  0.000000  0.191197
       3    No      0.231125  0.000000  0.000000  0.092162
            Yes     0.000000  0.000000  0.000000  0.257941
       4    No      0.000000  0.000000  0.000000  0.161573
            Yes     0.000000  0.000000  0.000000  0.186592
       5    No      0.000000  0.000000  0.000000  0.138160
       6    No      0.000000  0.000000  0.000000  0.211191

## 교차표

In [120]:
from io import StringIO

data = """Sample Nationality Handedenss
            1 USA Right-handed
            2 Japan Left-handed
            3 USA Right-handed
            4 Japan Right-handed
            5 Japan Left-handed
            6 Japan Right-handed
            7 USA Right-handed
            8 USA Left-handed
            9 Japan Right-handed
            10 USA Right-handed"""


data = pd.read_table(StringIO(data), sep='\s+')
data

,Sample,Nationality,Handedenss
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [121]:
pd.crosstab(data['Nationality'], data['Handedenss'], margins = True)

Handedenss,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [123]:
pd.crosstab([tips['time'], tips['day']], tips['smoker'], margins  = True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244